In [40]:
from bs4 import BeautifulSoup
from loguru import logger
import time
import numpy as np
from selenium import webdriver


class MediumCrawler:
    def __init__(self, scroll_limit: int = 5) -> None:
        options = webdriver.ChromeOptions()

        # options.add_argument("--no-sandbox")
        # options.add_argument("--headless=new")
        # options.add_argument("--disable-dev-shm-usage")
        # options.add_argument("--log-level=3")
        # options.add_argument("--disable-popup-blocking")
        # options.add_argument("--disable-notifications")
        # options.add_argument("--disable-extensions")
        # options.add_argument("--disable-background-networking")
        # options.add_argument("--ignore-certificate-errors")
        # options.add_argument(f"--user-data-dir={mkdtemp()}")
        # options.add_argument(f"--data-path={mkdtemp()}")
        # options.add_argument(f"--disk-cache-dir={mkdtemp()}")
        # options.add_argument("--remote-debugging-port=9226")
        # options.add_argument(r"--profile-directory=Profile 2")

        self.scroll_limit = scroll_limit
        self.driver = webdriver.Chrome(
            options=options,
        )

    def scroll_page(self) -> None:
        """Scroll through the LinkedIn page based on the scroll limit."""
        current_scroll = 0
        last_height = self.driver.execute_script("return document.body.scrollHeight")
        while True:
            self.driver.execute_script(
                "window.scrollTo(0, document.body.scrollHeight);"
            )
            time.sleep(max(1 + np.random.uniform(1), np.random.normal(2, 2)))
            new_height = self.driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height or (
                self.scroll_limit and current_scroll >= self.scroll_limit
            ):
                break
            last_height = new_height
            current_scroll += 1

    def extract(self, link: str) -> None:
        logger.info(f"Starting scrapping Medium article: {link}")

        self.driver.get(link)
        self.scroll_page()

        soup = BeautifulSoup(self.driver.page_source, "html.parser")
        title = soup.find_all("h1", class_="pw-post-title")
        subtitle = soup.find_all("h2", class_="pw-subtitle-paragraph")

        data = {
            "Title": title[0].string if title else None,
            "Subtitle": subtitle[0].string if subtitle else None,
            "Content": soup.get_text(),
        }

        self.driver.close()
        logger.info(f"Successfully scraped and saved article: {link}")

        return {"platfrom": "medium", "content": data, "link": link, "author": "temp"}

In [39]:
scraper = MediumCrawler()

scraper.extract(
    "https://medium.com/@cobusgreyling/why-the-focus-has-shifted-from-ai-agents-to-agentic-workflows-51e4078d03c2"
)

2025-03-19 13:26:46.903 | INFO     | __main__:extract:54 - Starting scrapping Medium article: https://medium.com/@cobusgreyling/why-the-focus-has-shifted-from-ai-agents-to-agentic-workflows-51e4078d03c2
2025-03-19 13:26:55.516 | INFO     | __main__:extract:70 - Successfully scraped and saved article: https://medium.com/@cobusgreyling/why-the-focus-has-shifted-from-ai-agents-to-agentic-workflows-51e4078d03c2


{'platfrom': 'medium',
 'content': {'Title': 'Why The Focus Has Shifted from AI Agents to Agentic Workflows',
  'Subtitle': 'We find ourselves on a stairway from where Large Language Models were introduced to AI Agents with human like digital interactions. But…',
  'Content': "Why The Focus Has Shifted from AI Agents to Agentic Workflows | by Cobus Greyling | Feb, 2025 | MediumOpen in appSign upSign inWriteSign upSign inHomeLibraryStoriesStatsTop highlight1Why The Focus Has Shifted from AI Agents to Agentic WorkflowsWe find ourselves on a stairway from where Large Language Models were introduced to AI Agents with human like digital interactions. But…Cobus Greyling·Follow5 min read·Feb 5, 20251.5K45ListenShare…there has a shift when it comes to commercial implementations with focus moving form AI Agents in favor of Agentic Workflows/Data Synthesis.Why is the focus moving away from AI Agents (for now)?Companies like Salesforce and Service made hard pivots to AI Agents, however, the stark

In [5]:
from pymongo import MongoClient

CLIENT: MongoClient = MongoClient("localhost", 27017)
DB = CLIENT.antonio
PEOPLE = DB.people
DOCUMENTS = DB.documents

In [9]:
for people in DOCUMENTS.find():
    print(people)

{'_id': ObjectId('67dade482ba62af1a5564a88'), 'platform': 'medium', 'content': {'Title': 'An End-to-End Framework for Production-Ready LLM Systems by Building Your LLM Twin', 'Subtitle': 'From data gathering to productionizing LLMs using LLMOps good practices.', 'Content': "End-to-End Framework for Production-Ready LLMs | Decoding MLOpen in appSign upSign inWriteSign upSign inHomeLibraryStoriesStatsDecoding MLHomeAbout·Follow publicationBattle-tested content on designing, coding, and deploying production-grade ML & MLOps systems. The hub for continuous learning on ML system design, ML engineering, MLOps, large language models (LLMs), and computer vision (CV).Follow publicationTop highlightFeaturedLLM Twin Course: Building Your Production-Ready AI ReplicaAn End-to-End Framework for Production-Ready LLM Systems by Building Your LLM TwinFrom data gathering to productionizing LLMs using LLMOps good practices.Paul Iusztin·FollowPublished inDecoding ML·16 min read·Mar 16, 20242.2K14ListenSha